In [1]:
## Import Module Requirements:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential ## !! The Model being used
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import itertools
import random

In [2]:
## Step 1: Define Problem
#########################
## Problem: Build a next word predictor
## Goal: Predict the next word in a sequence
#####################################################
## Outputs for Measuring Quality of Model Validation:
### TRUE Positive: Validation Data Label = TRUE;  Machine Learning Label Output = TRUE 
## FALSE Positive: Validation Data Label = FALSE; Machine Learning Label Output = TRUE 
### TRUE Negative: Validation Data Label = FALSE; Machine Learning Label Output = FALSE 
## FALSE Negative: Validation Data Label = TRUE;  Machine Learning Label Output = FALSE 

In [3]:
## Step 2: Collect & Split Dataset
##################################
import text_preprocessor # Import Variables via Files from Directory
from text_preprocessor import training_text # Import Variables via Files from Directory
##################################
## Example text from Training Data
print(training_text[0:100])

['theory', 'and', 'practice', 'of', 'piano', 'construction', 'by', 'william', 'b', 'white', '--', 'a', 'project', 'gutenberg', 'ebook', 'the', 'project', 'gutenberg', 'ebook', 'of', 'theory', 'and', 'practice', 'of', 'piano', 'construction', 'by', 'william', 'b', 'white', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restriction', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 'reuse', 'it', 'under', 'the', 'term', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'wwwgutenbergorglicense', 'title', 'theory', 'and', 'practice', 'of', 'piano', 'construction', 'with', 'a', 'detailed', 'practical', 'method', 'for', 'tuning', 'author', 'william', 'b', 'white', 'release', 'date', 'june', 'ebook', 'language', 'english', 'character', 'set']


In [4]:
## Get {keyword:id} Pairs
keyword_id_dict = {}
keyword_id = 1
for keyword in range(len(training_text)):
    if training_text[keyword] not in keyword_id_dict:
        keyword_id_dict[training_text[keyword]] = keyword_id
        keyword_id += 1

print(dict(itertools.islice(keyword_id_dict.items(), 30)))

{'theory': 1, 'and': 2, 'practice': 3, 'of': 4, 'piano': 5, 'construction': 6, 'by': 7, 'william': 8, 'b': 9, 'white': 10, '--': 11, 'a': 12, 'project': 13, 'gutenberg': 14, 'ebook': 15, 'the': 16, 'this': 17, 'is': 18, 'for': 19, 'use': 20, 'anyone': 21, 'anywhere': 22, 'at': 23, 'no': 24, 'cost': 25, 'with': 26, 'almost': 27, 'restriction': 28, 'whatsoever': 29, 'you': 30}


In [5]:
## Get a list of Text Sequences
text_sequences = []
sequence_training_len = 4
for keyword in range(sequence_training_len, len(training_text)):
    sequence = training_text[keyword - sequence_training_len:keyword]
    text_sequences.append(sequence)

print(text_sequences[0:30])

[['theory', 'and', 'practice', 'of'], ['and', 'practice', 'of', 'piano'], ['practice', 'of', 'piano', 'construction'], ['of', 'piano', 'construction', 'by'], ['piano', 'construction', 'by', 'william'], ['construction', 'by', 'william', 'b'], ['by', 'william', 'b', 'white'], ['william', 'b', 'white', '--'], ['b', 'white', '--', 'a'], ['white', '--', 'a', 'project'], ['--', 'a', 'project', 'gutenberg'], ['a', 'project', 'gutenberg', 'ebook'], ['project', 'gutenberg', 'ebook', 'the'], ['gutenberg', 'ebook', 'the', 'project'], ['ebook', 'the', 'project', 'gutenberg'], ['the', 'project', 'gutenberg', 'ebook'], ['project', 'gutenberg', 'ebook', 'of'], ['gutenberg', 'ebook', 'of', 'theory'], ['ebook', 'of', 'theory', 'and'], ['of', 'theory', 'and', 'practice'], ['theory', 'and', 'practice', 'of'], ['and', 'practice', 'of', 'piano'], ['practice', 'of', 'piano', 'construction'], ['of', 'piano', 'construction', 'by'], ['piano', 'construction', 'by', 'william'], ['construction', 'by', 'william', 

In [6]:
## Convert list of Text Sequences into its numerical keyword,id Pair
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences) # Updates internal vocabulary based on a list of texts.
sequences = tokenizer.texts_to_sequences(text_sequences) # Transforms each text in texts to a sequence of integers.
print(sequences[0:50])

[[453, 4, 152, 2], [4, 152, 2, 168], [152, 2, 168, 77], [2, 168, 77, 14], [168, 77, 14, 992], [77, 14, 992, 121], [14, 992, 121, 746], [992, 121, 746, 162], [121, 746, 162, 5], [746, 162, 5, 80], [162, 5, 80, 273], [5, 80, 273, 644], [80, 273, 644, 1], [273, 644, 1, 80], [644, 1, 80, 273], [1, 80, 273, 644], [80, 273, 644, 2], [273, 644, 2, 453], [644, 2, 453, 4], [2, 453, 4, 152], [453, 4, 152, 2], [4, 152, 2, 168], [152, 2, 168, 77], [2, 168, 77, 14], [168, 77, 14, 992], [77, 14, 992, 121], [14, 992, 121, 746], [992, 121, 746, 12], [121, 746, 12, 644], [746, 12, 644, 6], [12, 644, 6, 16], [644, 6, 16, 1], [6, 16, 1, 163], [16, 1, 163, 2], [1, 163, 2, 747], [163, 2, 747, 1452], [2, 747, 1452, 24], [747, 1452, 24, 65], [1452, 24, 65, 748], [24, 65, 748, 4], [65, 748, 4, 15], [748, 4, 15, 645], [4, 15, 645, 65], [15, 645, 65, 1733], [645, 65, 1733, 2098], [65, 1733, 2098, 111], [1733, 2098, 111, 29], [2098, 111, 29, 432], [111, 29, 432, 9], [29, 432, 9, 122]]


In [7]:
## Get a list of empty sequences
n_sequences = np.empty([len(sequences), sequence_training_len], dtype='int32')
print(n_sequences)

[[          0           0   257718592           1]
 [     999081           0          -1          -1]
 [ 2002475924  2123511579  -394174497  1218405217]
 ...
 [-1711472021 -1339695325  -309812460 -1236975977]
 [ 1449989897  1643624775 -1309731435 -1063851751]
 [ 1099052232  -884821824  1580753708   858819792]]


In [8]:
## Prep the training data
for sequence in range(len(sequences)):
    n_sequences[sequence] = sequences[sequence]
train_inputs = n_sequences[:,:-1] # Gets every keyword in a sequence except the last one (the keywords leading up to the TARGET)
train_targets = n_sequences[:,-1] # Gets the last keyword in a sequence (the TARGET)
vocabulary_size = len(tokenizer.word_counts) + 1 # vocabulary_size increased by 1 because of Padding
train_targets = to_categorical(train_targets, num_classes = vocabulary_size) # Converts a class vector (integers) to binary class matrix
sequence_len = train_inputs.shape[1] # The number of keywords in a sequence from the training data

In [9]:
## Step 3: Train Model
#################
## Prep the Model
model = Sequential()
model.add(Embedding(vocabulary_size, sequence_len, input_length=sequence_len))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))

In [10]:
## Train the Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse']) # Compile defines the loss function, the optimizer and the metrics
model.fit(train_inputs, train_targets, epochs=500, verbose=1) ## Fit is used for training the model with the provided inputs and targets

Epoch 1/500
1918/1918 [==============================] - 23s 10ms/step - loss: 6.3376 - accuracy: 0.1025 - mse: 2.0791e-04
Epoch 2/500
1918/1918 [==============================] - 15s 8ms/step - loss: 6.0227 - accuracy: 0.1050 - mse: 2.0628e-04
Epoch 3/500
1918/1918 [==============================] - 14s 7ms/step - loss: 5.8118 - accuracy: 0.1332 - mse: 2.0277e-04
Epoch 4/500
1918/1918 [==============================] - 14s 7ms/step - loss: 5.6444 - accuracy: 0.1405 - mse: 2.0139e-04
Epoch 5/500
1918/1918 [==============================] - 17s 9ms/step - loss: 5.5221 - accuracy: 0.1441 - mse: 2.0047e-04
Epoch 6/500
1918/1918 [==============================] - 16s 9ms/step - loss: 5.4272 - accuracy: 0.1472 - mse: 1.9967e-04
Epoch 7/500
1918/1918 [==============================] - 16s 8ms/step - loss: 5.3357 - accuracy: 0.1497 - mse: 1.9882e-04
Epoch 8/500
1918/1918 [==============================] - 15s 8ms/step - loss: 5.2361 - accuracy: 0.1545 - mse: 1.9788e-04
Epoch 9/500
1918/1918 [

1918/1918 [==============================] - 14s 7ms/step - loss: 3.5073 - accuracy: 0.2729 - mse: 1.8001e-04
Epoch 68/500
1918/1918 [==============================] - 14s 7ms/step - loss: 3.5031 - accuracy: 0.2750 - mse: 1.7987e-04
Epoch 69/500
1918/1918 [==============================] - 14s 7ms/step - loss: 3.4873 - accuracy: 0.2760 - mse: 1.7954e-04
Epoch 70/500
1918/1918 [==============================] - 14s 7ms/step - loss: 3.4825 - accuracy: 0.2759 - mse: 1.7950e-04
Epoch 71/500
1918/1918 [==============================] - 14s 7ms/step - loss: 3.4746 - accuracy: 0.2771 - mse: 1.7925e-04
Epoch 72/500
1918/1918 [==============================] - 14s 7ms/step - loss: 3.4671 - accuracy: 0.2772 - mse: 1.7898e-04
Epoch 73/500
1918/1918 [==============================] - 14s 7ms/step - loss: 3.4600 - accuracy: 0.2797 - mse: 1.7886e-04
Epoch 74/500
1918/1918 [==============================] - 14s 7ms/step - loss: 3.4501 - accuracy: 0.2821 - mse: 1.7864e-04
Epoch 75/500
1918/1918 [=====

1918/1918 [==============================] - 15s 8ms/step - loss: 3.1451 - accuracy: 0.3246 - mse: 1.7032e-04
Epoch 134/500
1918/1918 [==============================] - 15s 8ms/step - loss: 3.1415 - accuracy: 0.3259 - mse: 1.7008e-04
Epoch 135/500
1918/1918 [==============================] - 15s 8ms/step - loss: 3.1396 - accuracy: 0.3254 - mse: 1.7021e-04
Epoch 136/500
1918/1918 [==============================] - 15s 8ms/step - loss: 3.1352 - accuracy: 0.3257 - mse: 1.7001e-04
Epoch 137/500
1918/1918 [==============================] - 15s 8ms/step - loss: 3.1326 - accuracy: 0.3255 - mse: 1.6998e-04
Epoch 138/500
1918/1918 [==============================] - 15s 8ms/step - loss: 3.1277 - accuracy: 0.3278 - mse: 1.6966e-04
Epoch 139/500
1918/1918 [==============================] - 15s 8ms/step - loss: 3.1249 - accuracy: 0.3271 - mse: 1.6970e-04
Epoch 140/500
1918/1918 [==============================] - 15s 8ms/step - loss: 3.1225 - accuracy: 0.3276 - mse: 1.6951e-04
Epoch 141/500
1918/191

1918/1918 [==============================] - 15s 8ms/step - loss: 2.9493 - accuracy: 0.3543 - mse: 1.6416e-04
Epoch 200/500
1918/1918 [==============================] - 15s 8ms/step - loss: 2.9565 - accuracy: 0.3523 - mse: 1.6446e-04
Epoch 201/500
1918/1918 [==============================] - 15s 8ms/step - loss: 2.9499 - accuracy: 0.3530 - mse: 1.6408e-04
Epoch 202/500
1918/1918 [==============================] - 15s 8ms/step - loss: 2.9496 - accuracy: 0.3538 - mse: 1.6413e-04
Epoch 203/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.9436 - accuracy: 0.3544 - mse: 1.6395e-04
Epoch 204/500
1918/1918 [==============================] - 15s 8ms/step - loss: 2.9424 - accuracy: 0.3547 - mse: 1.6387e-04
Epoch 205/500
1918/1918 [==============================] - 15s 8ms/step - loss: 2.9377 - accuracy: 0.3574 - mse: 1.6360e-04
Epoch 206/500
1918/1918 [==============================] - 15s 8ms/step - loss: 2.9399 - accuracy: 0.3542 - mse: 1.6383e-04
Epoch 207/500
1918/191

1918/1918 [==============================] - 14s 7ms/step - loss: 2.8331 - accuracy: 0.3725 - mse: 1.6014e-04
Epoch 266/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.8250 - accuracy: 0.3735 - mse: 1.5995e-04
Epoch 267/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.8216 - accuracy: 0.3730 - mse: 1.5990e-04
Epoch 268/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.8231 - accuracy: 0.3732 - mse: 1.5994e-04
Epoch 269/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.8215 - accuracy: 0.3739 - mse: 1.5985e-04
Epoch 270/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.8217 - accuracy: 0.3727 - mse: 1.5997e-04
Epoch 271/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.8175 - accuracy: 0.3761 - mse: 1.5962e-04
Epoch 272/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.8194 - accuracy: 0.3746 - mse: 1.5990e-04
Epoch 273/500
1918/191

1918/1918 [==============================] - 14s 7ms/step - loss: 2.7400 - accuracy: 0.3873 - mse: 1.5711e-04
Epoch 332/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.7409 - accuracy: 0.3859 - mse: 1.5717e-04
Epoch 333/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.7364 - accuracy: 0.3868 - mse: 1.5693e-04
Epoch 334/500
1918/1918 [==============================] - 13s 7ms/step - loss: 2.7350 - accuracy: 0.3865 - mse: 1.5690e-04
Epoch 335/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.7307 - accuracy: 0.3874 - mse: 1.5670e-04
Epoch 336/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.7356 - accuracy: 0.3869 - mse: 1.5692e-04
Epoch 337/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.7295 - accuracy: 0.3878 - mse: 1.5680e-04
Epoch 338/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.7361 - accuracy: 0.3858 - mse: 1.5717e-04
Epoch 339/500
1918/191

1918/1918 [==============================] - 14s 7ms/step - loss: 2.6658 - accuracy: 0.3982 - mse: 1.5454e-04
Epoch 398/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6724 - accuracy: 0.3960 - mse: 1.5482e-04
Epoch 399/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6753 - accuracy: 0.3974 - mse: 1.5484e-04
Epoch 400/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6744 - accuracy: 0.3969 - mse: 1.5477e-04
Epoch 401/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6721 - accuracy: 0.3973 - mse: 1.5485e-04
Epoch 402/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6668 - accuracy: 0.3974 - mse: 1.5445e-04
Epoch 403/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6641 - accuracy: 0.3981 - mse: 1.5446e-04
Epoch 404/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6691 - accuracy: 0.3993 - mse: 1.5464e-04
Epoch 405/500
1918/191

1918/1918 [==============================] - 13s 7ms/step - loss: 2.6156 - accuracy: 0.4071 - mse: 1.5274e-04
Epoch 464/500
1918/1918 [==============================] - 13s 7ms/step - loss: 2.6236 - accuracy: 0.4056 - mse: 1.5306e-04
Epoch 465/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6084 - accuracy: 0.4084 - mse: 1.5247e-04
Epoch 466/500
1918/1918 [==============================] - 13s 7ms/step - loss: 2.6149 - accuracy: 0.4071 - mse: 1.5286e-04
Epoch 467/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6208 - accuracy: 0.4057 - mse: 1.5293e-04
Epoch 468/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6068 - accuracy: 0.4082 - mse: 1.5237e-04
Epoch 469/500
1918/1918 [==============================] - 13s 7ms/step - loss: 2.6163 - accuracy: 0.4056 - mse: 1.5286e-04
Epoch 470/500
1918/1918 [==============================] - 14s 7ms/step - loss: 2.6104 - accuracy: 0.4083 - mse: 1.5266e-04
Epoch 471/500
1918/191

In [11]:
## Step 4: Debug & Tune Model
## Debug & Tune Model: Validate Model using the Debugging Dataset
#### Review Machine Learning Label Output vs Debugging Dataset Label
#### IF inspired THEN fix issues (dataset, hyperparameters, etc.)
validation_text = 'if the tuned'
validation_text = validation_text.strip().lower()
print('First 3 words in a sequence:')
print(validation_text)
print('\n')

encoded_text = tokenizer.texts_to_sequences([validation_text])[0] ## Converts validation text to keyword_id
pad_encoded = pad_sequences([encoded_text], maxlen=sequence_len, truncating='pre') ## Converts keyword_ids to array

for keyword in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    predicted_next_word = tokenizer.index_word[keyword]
    print("Suggested next word: '{0}'".format(predicted_next_word))

First 3 words in a sequence:
if the tuned


1/1 [==============================] - 2s 2s/step
Suggested next word: 'is'
Suggested next word: 'and'
Suggested next word: 'note'
